# Type B evaluatie: NLP en Unsupervised learning

In deze opgave ga je werken met een dataset over verschillende films.
In deze opgave ga je proberen de winst van een film te voorspellen op basis van een dataset die je reeds gekregen hebt en de beschrijving van de films.
Daarna ga je kijken met principal component analysis om de dataset te reduceren met zoveel mogelijk informatie over te houden.
Ten slotte gaan we kijken om films te clusteren om genres op een automatische manier toe te kennen.

## Opstellen van de dataset

Hieronder krijg je eerst de code voor het downloaden en opstellen van de dataset.
Voer deze code uit om de dataset te downloaden en in te laden voor je aan de volgende stappen begint.

In [1]:
import opendatasets as od
import pandas as pd
import numpy as np

od.download("https://www.kaggle.com/datasets/utkarshx27/movies-dataset")
df_movies = pd.read_csv("./movies-dataset/movie_dataset.csv")

df_movies_released = df_movies[df_movies.status == "Released"]

# stap 1/2/3
df_origin_with_budget = df_movies_released[df_movies_released.budget > 0]
df = df_origin_with_budget[["budget", "revenue", "vote_average"]].copy()

# stap 3
df["winst"] = (df.revenue - df.budget) / df.budget

# stap 4
df["language_ord_enc"] = pd.factorize(df_origin_with_budget.original_language)[0]

# stap 5
one_hots = df_origin_with_budget['genres'].str.get_dummies(sep=' ')
df = pd.concat([df, one_hots], axis=1)
df["Science Fiction"] = df.Science
df.drop(["Science", "Fiction"], axis=1, inplace=True)

# stap 6
def enc_runtime(value):
    if value == 0:
        return "unknown"
    elif value <= 30:
        return "kortspeelfilm"
    else:
        return "langspeelfilm"
df["runtime_enc"] = df_origin_with_budget.runtime.apply(enc_runtime)

# stap 7a,b,c
df_origin_with_budget.tagline = df_origin_with_budget.tagline.fillna('')
df_origin_with_budget.cast = df_origin_with_budget.cast.fillna('')
df_origin_with_budget.crew = df_origin_with_budget.crew.fillna('')
df["tagline_num_words"] = df_origin_with_budget.tagline.str.split(' ').apply(len)
df["cast_size"] = df_origin_with_budget.cast.str.split(' ').apply(len)
df["crew_size"] = df_origin_with_budget.crew.str.split('},').apply(len)

#stap 8
df = df.drop(["budget", "revenue"], axis=1)

display(df)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Dataset URL: https://www.kaggle.com/datasets/utkarshx27/movies-dataset


100%|██████████| 5.13M/5.13M [00:00<00:00, 7.39MB/s]


C:\Users\jens.baetens3\AppData\Local\Temp\ipykernel_10716\230137448.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_origin_with_budget.tagline = df_origin_with_budget.tagline.fillna('')
C:\Users\jens.baetens3\AppData\Local\Temp\ipykernel_10716\230137448.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_origin_with_budget.cast = df_origin_with_budget.cast.fillna('')
C:\Users\jens.baetens3\AppData\Local\Temp\ipykernel_10716\230137448.py:39: SettingWithCopyWarning: 
A value is trying to be set on

,vote_average,winst,language_ord_enc,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,...,Romance,TV,Thriller,War,Western,Science Fiction,runtime_enc,tagline_num_words,cast_size,crew_size
0,7.2,10.763566,0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,langspeelfilm,5,10,153
1,6.9,2.203333,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,langspeelfilm,9,10,32
2,6.3,2.594590,0,1,1,0,0,1,0,0,...,0,0,0,0,0,0,langspeelfilm,5,10,155
3,7.6,3.339756,0,1,0,0,0,1,0,1,...,0,0,1,0,0,0,langspeelfilm,3,10,217
4,6.1,0.092843,0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,langspeelfilm,7,11,132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4791,2.0,-1.000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,langspeelfilm,5,9,4
4792,7.4,3.950000,1,0,0,0,0,1,0,0,...,0,0,1,0,0,0,langspeelfilm,3,10,2
4796,6.9,59.680000,0,0,0,0,0,0,0,1,...,0,0,1,0,0,1,langspeelfilm,6,10,6
4798,6.6,8.276909,4,1,0,0,0,1,0,0,...,0,0,1,0,0,0,langspeelfilm,12,11,11


## NLP

Voor deze opgave moet nog de beschrijving van de film toegevoegd worden. 
De beschrijving van deze film staat in de **overview** kolom in het dataframe in de parameter df_movies_released.
Gebruik NLP technieken om de extra features toe te voegen aan het dataframe **df**.
Maak twee dataframes aan om dit te verwezenlijken waarbij de extra features toegevoegd worden als:
* een multi-hot encoding met de [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) dat telt hoeveel keer elk woord voorkomt
* een tf-idf encoding met de [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) dat meer waarde toekent aan zeldzamere woorden.

Maak nu een pijplijn om de winst van een film te voorspellen.
In de oplossing moeten zeker de volgende stappen aanwezig zijn:
* Splits de dataset in features en target
* Maak een preprocessor stap aan in de pijplijn die ervoor zorgt dat alle inputs numeriek zijn. Let ook dat de correcte kolommen geschaald zijn.
* Gebruik een functie waaraan je de features (training en test), labels (training en test), de preprocessor, de regressor en een lijst van parameters aan meegeeft. Deze functie stelt de pipeline samen uit de preprocessor en de regressor, traint het model, zoekt naar de beste parameters en evalueert het model (door een aantal metrieken uit te printen)
* Zorg ervoor dat je de functie gebruikt om de twee opgestelde dataframes te trainen met een Naive Bayes model.

Let hierbij zeker op de best practices voor het trainen van een model die gezien zijn in de les.

Antwoord ook zeker op de volgende vragen:
* Heb je bij 1 van de bekomen modellen gemerkt dat er overfitting/underfitting aanwezig is? Verklaar je antwoord?
* Welke encoding van de tekst geeft het beste resultaat (multi-hot of tf-idf)? Verklaar je antwoord en verwijs naar hoe de technieken werken indien nodig.

**Antwoord:**

In [49]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import FunctionTransformer

In [53]:
df['beschrijving'] = df_movies_released.overview
display(df.head())

text_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('reshape', FunctionTransformer(lambda x: x.ravel())),  # Zorg voor een eendimensionale array
    ('vectorizer', CountVectorizer())
])

preprocessor = ColumnTransformer(transformers=[
    ('text', text_pipeline , ['beschrijving'])
])

pipeline = Pipeline([
    ('preprocessor', preprocessor)
])

pipeline.fit_transform(df).todense().shape

C:\Users\jens.baetens3\AppData\Local\Temp\ipykernel_10716\3846020442.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['beschrijving'] = df_movies_released.overview


,vote_average,winst,language_ord_enc,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,...,TV,Thriller,War,Western,Science Fiction,runtime_enc,tagline_num_words,cast_size,crew_size,beschrijving
0,7.2,10.763566,0,1,1,0,0,0,0,0,...,0,0,0,0,1,langspeelfilm,5,10,153,"In the 22nd century, a paraplegic Marine is di..."
1,6.9,2.203333,0,1,1,0,0,0,0,0,...,0,0,0,0,0,langspeelfilm,9,10,32,"Captain Barbossa, long believed to be dead, ha..."
2,6.3,2.594590,0,1,1,0,0,1,0,0,...,0,0,0,0,0,langspeelfilm,5,10,155,A cryptic message from Bond’s past sends him o...
3,7.6,3.339756,0,1,0,0,0,1,0,1,...,0,1,0,0,0,langspeelfilm,3,10,217,Following the death of District Attorney Harve...
4,6.1,0.092843,0,1,1,0,0,0,0,0,...,0,0,0,0,1,langspeelfilm,7,11,132,"John Carter is a war-weary, former military ca..."


(3760, 18434)

## Unsupervised learning - Principal Component Analysis

Voer nu PCA uit om de 5 features te zoeken die het meeste impact hebben op de voorspellingen.

Beantwoord hieronder de volgende vragen:
* Wat zijn de 5 sterkste features?
* Hoeveel procent van de variabiliteit wordt verklaard door deze features?
* Beschrijf beknopt hoe PCA werkt en hoe daaruit de sterkste features gehaald kunnen worden.

**Antwoord:**


## Unsupervised learning - Clustering

In het laatste deel gaan we proberen gelijkaardige films te groeperen in clusters.
Verwijder hiervoor eerst de features die te maken hebben met de genres.

Gebruik nu het K-Means en Mean-shift clustering algoritme voor clustering uit te voeren.
Ga hierbij ook op zoek naar het ideale aantal clusters.

Ten slotte, gebruik het K-means algoritme om clustering uit te voeren waarbij K gelijk is aan het aantal verschillende genres.

Beantwoord daarna uit bovenstaande behaalde resultaten daarna de volgende vragen:
* Hoe noemt de methode om het beste aantal clusters te bepalen? Hoe werkt deze methode?
* Hebben beide algoritmes het beste aantal? Kan je een inschatting maken of dit aantal ergens mee overeenkomt.
* Bij het uitvoeren van het K-means algoritme met K het aantal genres, bekijk een aantal films in een bepaalde cluster. Hebben deze films gelijkaardige genres of niet? Is dit mogelijk of wijst dit op een fout in de dataset?


In [ ]:
# K-means - zoektocht naar optimaal aantal clusters

In [ ]:
# Mean shift - zoektocht naar optimaal aantal clusters

In [ ]:
# K-means - waar K het aantal genres is

In [ ]:
# extra berekeningen voor de vragen te beantwoorden